In [1]:
# The code was removed by Watson Studio for sharing.

# Predicting English Premier League Matches

By: Yi Xuan Sim

## Model Definition, Model Training and Model Evaluation

Loading the dataset with new features

In [2]:
# The code was removed by Watson Studio for sharing.

,HomeTeam,AwayTeam,FTR,home_attack_score_analysis,away_attack_score_analysis,home_defense_score_analysis,away_defense_score_analysis,home_attack_efficiency_analysis,away_attack_efficiency_analysis,home_inv_defense_efficiency_analysis,away_inv_defense_efficiency_analysis,diff_attack,diff_defense,overall_score
0,Aston Villa,Bolton,D,-0.367044,-0.097585,-0.143126,0.377551,-0.241390,-0.463096,-0.722440,-0.621563,-0.269459,0.520677,0.251218
1,Everton,Man United,A,0.855378,2.162607,-0.614336,-2.168971,0.418593,0.998026,-0.438074,-1.517423,-1.307229,-1.554635,-2.861864
2,Fulham,Birmingham,D,0.076383,-0.544953,0.074727,0.420843,-0.483306,-1.066144,-0.439501,-1.018328,0.621337,0.346116,0.967453
3,Man City,West Brom,D,2.167449,-0.614544,-1.192720,-0.411329,1.120426,-0.396703,-0.398956,-0.544577,2.781993,0.781391,3.563383
4,Middlesbrough,Liverpool,D,-0.268881,1.670617,0.067824,-1.633356,-0.334656,0.369332,-0.166129,-0.043594,-1.939499,-1.701180,-3.640678


Looking at dataset, we have several features, and we are trying to predict FTR (Full Time Result). This is a supervised learning problem. As FTR is a categorical data type, we will use a classifier to predict the outcomes. We will use 2 models, a non-deep learning based model, and a deep learning model. 

### Non-deep learning model

For the non-deep learning model, we will use various classifiers, such as RandomForest Classifier, SVC, and Decision Tree Classifier to predict the outcome of the match. 

In [3]:
from sklearn.ensemble import RandomForestClassifier 
from sklearn.svm import SVC 
from sklearn.tree import DecisionTreeClassifier 

In [4]:
from sklearn.cross_validation import train_test_split #this is required to split the data into training and test data

#data is split into training and testing datasets in 70:30 ratio
x_all = df_data_1.drop(['HomeTeam','AwayTeam','FTR'], axis=1)
y_all = df_data_1['FTR']
x_train, x_test, y_train, y_test = train_test_split(x_all, y_all, test_size = 0.30, random_state = 99)

/usr/local/src/conda3_runtime/home/envs/DSX-Python35-Spark/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


### Model Training

In [5]:
#Introducing each classifier with own variables to make the next step in training easier by doing it all at once
clf1 = RandomForestClassifier()
clf2 = SVC()
clf3 = DecisionTreeClassifier()

In [6]:
#Starting to training the data and cross validate

y_pred1 = clf1.fit(x_train,y_train).predict(x_test) #fitting the training data with RandomForestClassifier
y_pred2 = clf2.fit(x_train,y_train).predict(x_test) #fitting the training data with SVC
y_pred3 = clf3.fit(x_train,y_train).predict(x_test) #fitting the training data with DecisionTreeClassifier

### Model Evaluation

We use 10 fold cross-validation to evaluate the model

In [7]:
from sklearn.metrics import accuracy_score #to calculate the number of correct prediction against true value
from sklearn.model_selection import cross_val_score #to cross validate and acts as a scoring system for classifier

acc1 = accuracy_score(y_pred1,y_test)
scores1 = cross_val_score(clf1, x_train, y_train, cv=10)

acc2 = accuracy_score(y_pred2,y_test)
scores2 = cross_val_score(clf2, x_train, y_train, cv=10)

acc3 = accuracy_score(y_pred3,y_test)
scores3 = cross_val_score(clf3, x_train, y_train, cv=10)

#Getting the mean scores for the cross validation
scores1_mean = scores1.mean()
scores2_mean = scores2.mean()
scores3_mean = scores3.mean()

print('By predicting using all features')
print("Accuracy",'%.2f'%acc1, "Score", '%.3f'%scores1_mean,"RandomForestClassifier")
print("Accuracy",'%.2f'%acc2, "Score", '%.3f'%scores2_mean,"SVC")
print("Accuracy",'%.2f'%acc3, "Score", '%.3f'%scores3_mean,"DecisionTreeClassifier")

By predicting using all features
Accuracy 0.47 Score 0.472 RandomForestClassifier
Accuracy 0.52 Score 0.554 SVC
Accuracy 0.46 Score 0.463 DecisionTreeClassifier


From the results, we can see that the Random Forest Classifier and Decision Tree Classifier has an accuracy of less than 50%, while SVC provides an accuracy of 52%. We shall now use Adaboost to investigate whether it improves performance. 

By default, Adaboost uses RandomForest as the base estimator. Boosting works best when paired with a non-linear model. We choose Random Forest as it is a non-linear model, while SVC is a linear model. 

### Ensemble Methods - Using Adaboost

In [8]:
from sklearn.ensemble import AdaBoostClassifier
#svc=SVC(probability=True, kernel='linear')
clf_ada = AdaBoostClassifier(n_estimators=100, learning_rate=0.5)
model_ada = clf_ada.fit(x_train, y_train)
y_pred_ada = model_ada.predict(x_test)


In [9]:
#Adaboost with RandomForests
acc_ada = accuracy_score(y_pred_ada,y_test)
scores_ada = cross_val_score(clf_ada, x_train, y_train, cv=10)

#Getting the mean scores for the cross validation
scores_ada_mean = scores_ada.mean()

print('Prediction using Adaboost')
print("Accuracy",'%.2f'%acc_ada, "Score", '%.3f'%scores_ada_mean)

Prediction using Adaboost
Accuracy 0.52 Score 0.556


### Deep Learning Model

Using neural networks to classify the outcome of a match. Here, we are using a multi-layer perceptron (MLP) for multi-class classification

### Iteration 1: Feature Creation
Neural networks only accept numerical inputs and outputs, thus we use one hot encoding to transform the labels

We first tranform the labels into integers. Later we will use keras to_categorical to transform into binary values.

The encoding is as follows: <br/>
Win by away team = 0 <br/>
Draw = 1 <br/>
Win by home team = 2 <br/>

In [10]:
full_time = df_data_1['FTR']
analysis_NN = df_data_1.copy()

from sklearn.preprocessing import LabelEncoder
# integer encode
label_encoder = LabelEncoder()
FT_encoded = label_encoder.fit_transform(full_time)
analysis_NN['FT_encoded'] = FT_encoded

#drop all non numerical values
analysis_NN = analysis_NN.drop(['HomeTeam','AwayTeam','FTR'], axis=1)
analysis_NN.head()

,home_attack_score_analysis,away_attack_score_analysis,home_defense_score_analysis,away_defense_score_analysis,home_attack_efficiency_analysis,away_attack_efficiency_analysis,home_inv_defense_efficiency_analysis,away_inv_defense_efficiency_analysis,diff_attack,diff_defense,overall_score,FT_encoded
0,-0.367044,-0.097585,-0.143126,0.377551,-0.241390,-0.463096,-0.722440,-0.621563,-0.269459,0.520677,0.251218,1
1,0.855378,2.162607,-0.614336,-2.168971,0.418593,0.998026,-0.438074,-1.517423,-1.307229,-1.554635,-2.861864,0
2,0.076383,-0.544953,0.074727,0.420843,-0.483306,-1.066144,-0.439501,-1.018328,0.621337,0.346116,0.967453,1
3,2.167449,-0.614544,-1.192720,-0.411329,1.120426,-0.396703,-0.398956,-0.544577,2.781993,0.781391,3.563383,1
4,-0.268881,1.670617,0.067824,-1.633356,-0.334656,0.369332,-0.166129,-0.043594,-1.939499,-1.701180,-3.640678,1


In [11]:
from keras.utils import to_categorical
x_all = analysis_NN.drop(['FT_encoded'], axis=1)
#to_categorical converts integers into binary values
y_all = to_categorical(analysis_NN['FT_encoded'])
print(analysis_NN.shape[0])

Using TensorFlow backend.


5150


We want to keep 20% of the data for manual cross validation, so we split the data into training and testing sets. <br/>
20% of 5150 is 1030 <br/>
5150 - 1030 is 4120 <br/>


In [12]:
x_train = x_all[:4120]
x_test = x_all[-1030:]
y_train = y_all[:4120]
y_test = y_all[-1030:]
x_train_v = x_train.values
x_test_v = x_test.values

### Deep Learning Model Training

Now, we will define and train a neural network with 4 layers. <br/>
In the first layer, we must specify the expected input data shape. Since we have 9 features, the input dimension is 9.  <br/>
The final layer has 3 nodes, as there are only 3 outputs (Away win, Draw, Home win). <br/>
We will use the categorical crossentropy loss function to evaluate our model performance. 


In [14]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import SGD
from keras import metrics

model = Sequential()
# Dense(64) is a fully-connected layer with 64 hidden units.
model.add(Dense(64, input_dim=11))
model.add(Activation('sigmoid'))
model.add(Dense(32, activation='sigmoid'))
model.add(Dense(10, activation='sigmoid'))
model.add(Dense(3, activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

model.fit(x_train_v, y_train,epochs=20, batch_size=128, validation_split=0.3)

Train on 2884 samples, validate on 1236 samples
Epoch 1/20
2884/2884 [==============================] - 0s - loss: 1.1208 - acc: 0.4227 - val_loss: 1.0719 - val_acc: 0.4442
Epoch 2/20
2884/2884 [==============================] - 0s - loss: 1.0505 - acc: 0.4771 - val_loss: 1.0686 - val_acc: 0.4442
Epoch 3/20
2884/2884 [==============================] - 0s - loss: 1.0488 - acc: 0.4771 - val_loss: 1.0634 - val_acc: 0.4442
Epoch 4/20
2884/2884 [==============================] - 0s - loss: 1.0457 - acc: 0.4771 - val_loss: 1.0629 - val_acc: 0.4442
Epoch 5/20
2884/2884 [==============================] - 0s - loss: 1.0434 - acc: 0.4771 - val_loss: 1.0599 - val_acc: 0.4442
Epoch 6/20
2884/2884 [==============================] - 0s - loss: 1.0407 - acc: 0.4771 - val_loss: 1.0571 - val_acc: 0.4442
Epoch 7/20
2884/2884 [==============================] - 0s - loss: 1.0377 - acc: 0.4771 - val_loss: 1.0533 - val_acc: 0.4442
Epoch 8/20
2884/2884 [==============================] - 0s - loss: 1.0348 - a

From the training results, we can see that the accuracy at the last epoch is 54%, while the cross validation accuracy is 53%. The performance is slightly better than the non deep learning based classifiers. We shall investigate how to improve the performance. 

In [15]:
y_pred = model.predict(x_test_v)

In [16]:
print(y_pred)

[[ 0.12992436  0.21971498  0.65036064]
 [ 0.3853341   0.26531795  0.34934795]
 [ 0.35837227  0.26998004  0.37164766]
 ..., 
 [ 0.23807357  0.2582016   0.50372487]
 [ 0.43744841  0.26197335  0.30057815]
 [ 0.24788913  0.25905833  0.49305257]]


In [31]:
import numpy as np
y_pred_int = np.argmax(y_pred, axis=1)
y_actual_int = np.argmax(y_test, axis=1)

#Encode back into ADH form
y_pred_encode = pd.cut(y_pred_int,3, labels=["A", "D", "H"])
y_actual_encode = pd.cut(y_actual_int,3, labels=["A", "D", "H"])

final_predict_NN = x_test.copy()
final_predict_NN['y_pred'] = y_pred_encode
final_predict_NN['y_actual'] = y_actual_encode
final_predict_NN.head()

,home_attack_score_analysis,away_attack_score_analysis,home_defense_score_analysis,away_defense_score_analysis,home_attack_efficiency_analysis,away_attack_efficiency_analysis,home_inv_defense_efficiency_analysis,away_inv_defense_efficiency_analysis,diff_attack,diff_defense,overall_score,y_pred,y_actual
4120,1.790389,-0.753725,-1.656020,-0.159273,0.202558,0.037929,-0.187167,-0.660861,2.544114,1.496748,4.040862,H,H
4121,-0.538829,0.909730,0.084051,-0.215602,-0.389536,2.447389,-0.550693,1.692358,-1.448559,-0.299653,-1.748212,A,A
4122,1.310207,2.162607,-1.076366,-2.168971,-0.216535,0.998026,-0.858941,-1.517423,-0.852399,-1.092606,-1.945005,H,H
4123,-0.675088,0.368281,0.112023,-1.208595,0.374926,-0.137045,1.434206,-0.827881,-1.043369,-1.320618,-2.363988,A,D
4124,2.316548,1.849993,-1.534492,-1.451840,0.589495,0.835882,-0.553050,-0.434561,0.466555,0.082652,0.549207,H,A


In [32]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_actual_encode, y_pred_encode, labels=["A", "D", "H"])

array([[174,   0, 130],
       [ 62,   0, 182],
       [ 76,   0, 406]])

In [33]:
#Accuracy Scoring
count = 0
if len(y_pred_encode) == len(y_actual_encode):
    for i in range(len(y_pred_encode)):
        if y_pred_encode[i] == y_actual_encode[i]:
             count += 1

print("Accuracy = " + str(count*1.0/1030))

Accuracy = 0.5631067961165048


Our Neural Network has a tendency to predict home wins and away wins, but not draws. This is because our current approach chooses the class with the highest probability.

For instance, if the predicted probabilities are [0.37, 0.28, 0.34] , the neural network will predict the class of probability 0.37, which is an away win. This is clearly incorrect, as the difference between a home win (0.34) and a away win (0.37) is small.

### Approach for predicting draws

We set a threshold value equal to 0.05. We then check for the difference in predicted probabilies between away win and home win. If the difference is less than the threshold value, then we predict a draw.

In [34]:
threshold = 0.05
count_1 = 0
for row in y_pred:
    diff = abs(row[2] - row[0])
    if (diff<threshold):
        y_pred_encode[count_1] = 'D'
    count_1 += 1

In [35]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_actual_encode, y_pred_encode, labels=["A", "D", "H"])

array([[168,  16, 120],
       [ 59,   9, 176],
       [ 72,  15, 395]])

In [36]:
#New Accuracy
count_2 = 0
if len(y_pred_encode) == len(y_actual_encode):
    for i in range(len(y_pred_encode)):
        if y_pred_encode[i] == y_actual_encode[i]:
             count_2 += 1

print("Accuracy = " + str(count_2*1.0/1030))

Accuracy = 0.5553398058252427


### Final Model Evaluation

In summary, for non deep learning based models, the SVC Classifier and the ensemble based method with Adaboost provides the highest accuracy at 52%. On the other hand, for a deep learning based neural network, the accuracy is around 54-55%. Although this may seem like a low figure, it turns out the world's most established football bookmakers, such as Bet365, have a betting accuracy of around 53%. In a football match, numerous factors, such as weather conditions, cannot be accounted for, giving rise to inaccuracies in the prediction. Besides, with some additional modifications, our neural network now has the ability to predict draws as well. Thus, we will choose the neural network model for use in predictions. 